# VacationPy
----

#### Note
* Keep an eye on your API usage. Use https://developers.google.com/maps/reporting/gmp-reporting as reference for how to monitor your usage and billing.

* Instructions have been included for each segment. You do not have to follow them exactly, but they are included to help you think through the steps.

In [1]:
# Dependencies and Setup
import matplotlib.pyplot as plt
import pandas as pd
import numpy as np
import requests
import gmaps
import os

# Import API key
from api_keys import g_key

### Store Part I results into DataFrame
* Load the csv exported in Part I to a DataFrame

In [2]:
## the cities from WeatherPy
weather_df = pd.read_csv("../output_data/cities.csv")
weather_df

,Unnamed: 0,City,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Country,Date
0,0,Saint Paul Harbor,57.7900,-152.4072,56.71,88,75,5.75,US,1661371707
1,1,Mataura,-46.1927,168.8643,32.49,86,100,2.53,NZ,1661371708
2,2,Ugoofaaru,5.6667,73.0000,77.74,84,100,13.53,MV,1661371708
3,3,Kralendijk,12.1500,-68.2667,87.78,66,0,19.57,BQ,1661371709
4,4,Castro,-24.7911,-50.0119,65.32,65,59,3.78,BR,1661371710
...,...,...,...,...,...,...,...,...,...,...
532,532,Pilot Butte,50.4668,-104.4178,72.66,49,44,6.73,CA,1661371966
533,533,Mumford,5.2618,-0.7590,73.15,90,4,5.64,GH,1661371967
534,534,Mata,40.6830,-7.4592,67.93,85,3,7.99,PT,1661371967
535,535,Fort-Shevchenko,44.5086,50.2630,78.19,37,0,14.88,KZ,1661371968


### Humidity Heatmap
* Configure gmaps.
* Use the Lat and Lng as locations and Humidity as the weight.
* Add Heatmap layer to map.

In [3]:
# Configure gmaps
gmaps.configure(api_key=g_key)

# Store latitude and longitude in locations
locations = weather_df[["Lat", "Lng"]]

# Fill NaN values and convert to float
humidity = weather_df["Humidity"].astype(float)


In [4]:
# Plot Heatmap
fig = gmaps.figure(center=[0,0], zoom_level=2)
max_intensity = np.max(humidity)

# Create heat layer
heat_layer = gmaps.heatmap_layer(locations, weights=humidity, 
                                 dissipating=False, max_intensity=100,
                                 point_radius=1)


# Add layer
fig.add_layer(heat_layer)

# Display figure
fig

Figure(layout=FigureLayout(height='420px'))

### Create new DataFrame fitting weather criteria
* Narrow down the cities to fit weather conditions.
* Drop any rows will null values.

In [5]:
## using loc and & filter the data based on the instructions: max temp lower than 80 but more than 70, wind speed less than 10mph, zero cloudiness
new_city_df = weather_df.loc[(weather_df["Wind Speed"] <= 10) & (weather_df["Cloudiness"] == 0) & \
                                   (weather_df["Max Temp"] >= 70) & (weather_df["Max Temp"] <= 80)].dropna()

new_city_df

,Unnamed: 0,City,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Country,Date
9,9,Port Elizabeth,-33.9180,25.5701,77.29,45,0,9.22,ZA,1661371612
83,83,Poum,-20.2333,164.0167,73.44,74,0,7.87,NC,1661371747
88,88,Vao,-22.6667,167.4833,72.39,75,0,7.40,NC,1661371749
108,108,Goderich,43.7501,-81.7165,74.91,63,0,7.18,CA,1661371572
161,161,Pacific Grove,36.6177,-121.9166,74.39,82,0,9.22,US,1661371784
176,176,Jerusalem,31.7690,35.2163,75.70,60,0,3.94,IL,1661371792
231,231,Mpanda,-6.3655,31.0409,71.17,38,0,0.94,TZ,1661371821
302,302,Koumac,-20.5667,164.2833,70.59,78,0,3.67,NC,1661371855
312,312,Constantine,36.3650,6.6147,76.41,57,0,6.91,DZ,1661371860
360,360,Mahon,39.8885,4.2658,77.56,78,0,2.30,ES,1661371883


### Hotel Map
* Store into variable named `hotel_df`.
* Add a "Hotel Name" column to the DataFrame.
* Set parameters to search for hotels with 5000 meters.
* Hit the Google Places API for each city's coordinates.
* Store the first Hotel result into the DataFrame.
* Plot markers on top of the heatmap.

In [6]:
## store only the columns suggested and add a column for Hotel Name
# Create a hotel_df
hotel_df = pd.DataFrame(new_city_df, columns=["City","Country", "Lat", "Lng"])

# Add a "Hotel Name" column to the DataFrame.
hotel_df["Hotel Name"] = ""

# Display the result
hotel_df

,City,Country,Lat,Lng,Hotel Name
9,Port Elizabeth,ZA,-33.9180,25.5701,
83,Poum,NC,-20.2333,164.0167,
88,Vao,NC,-22.6667,167.4833,
108,Goderich,CA,43.7501,-81.7165,
161,Pacific Grove,US,36.6177,-121.9166,
176,Jerusalem,IL,31.7690,35.2163,
231,Mpanda,TZ,-6.3655,31.0409,
302,Koumac,NC,-20.5667,164.2833,
312,Constantine,DZ,36.3650,6.6147,
360,Mahon,ES,39.8885,4.2658,


In [7]:
## set params like API Day 3 Activity 4
params = {
    "radius": 5000,
    "type": "hotel",
    "keyword": "hotel",
    "key": g_key
}

base_url = "https://maps.googleapis.com/maps/api/place/nearbysearch/json"

In [8]:
# NOTE: Do not change any of the code in this cell
# Using the template add the hotel marks to the heatmap
info_box_template = """
<dl>
<dt>Name</dt><dd>{Hotel Name}</dd>
<dt>City</dt><dd>{City}</dd>
<dt>Country</dt><dd>{Country}</dd>
</dl>
"""
# Store the DataFrame Row
# NOTE: be sure to update with your DataFrame name
hotel_info = [info_box_template.format(**row) for index, row in hotel_df.iterrows()]
locations = hotel_df[["Lat", "Lng"]]

In [9]:
# Use the lat/ng to identify hotels
for index, row in hotel_df.iterrows():
    
    # get city name, lat, lnt from df
    lat = row["Lat"]
    lng = row["Lng"]
    city_name = row["City"]
    
    # add keyword to params dict
    params["location"] = f"{lat},{lng}"

    # assemble url and make API request
    print(f"Retrieving Results for Index {index}: {city_name}.")
    response = requests.get(base_url, params=params).json()
    
    # extract results
    results = response['results']
    
    # save the hotel name to dataframe
    try:
        print(f"Closest hotel in {city_name} is {results[0]['name']}.")
        hotel_df.loc[index, "Hotel Name"] = results[0]['name']

    except (KeyError, IndexError):
        print("Missing Field. skipping...")
        
    print("-----------------------")

print("-------End of Search-------")

Retrieving Results for Index 9: Port Elizabeth.
Closest hotel in Port Elizabeth is Herms Restaurant and Boutique Hotel.
-----------------------
Retrieving Results for Index 83: Poum.
Missing Field. skipping...
-----------------------
Retrieving Results for Index 88: Vao.
Closest hotel in Vao is OURE TERA beach resort.
-----------------------
Retrieving Results for Index 108: Goderich.
Closest hotel in Goderich is Samuels Boutique Hotel.
-----------------------
Retrieving Results for Index 161: Pacific Grove.
Closest hotel in Pacific Grove is InterContinental the Clement Monterey, an IHG Hotel.
-----------------------
Retrieving Results for Index 176: Jerusalem.
Closest hotel in Jerusalem is Hashimi Hotel.
-----------------------
Retrieving Results for Index 231: Mpanda.
Closest hotel in Mpanda is White Rose Lodge.
-----------------------
Retrieving Results for Index 302: Koumac.
Closest hotel in Koumac is Karem Bay.
-----------------------
Retrieving Results for Index 312: Constantine.

In [10]:
# Add marker layer ontop of heat map
markers = gmaps.marker_layer(locations, info_box_content=hotel_info)
fig.add_layer(markers)

# Display figure
fig

Figure(layout=FigureLayout(height='420px'))